In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [2]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize Text

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [23]:
model = Sequential()
model.add(Dense(300, input_shape=(max_sequence_len-1,), activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 300)               4800      
                                                                 
 dense_28 (Dense)            (None, 800)               240800    
                                                                 
 dense_29 (Dense)            (None, 800)               640800    
                                                                 
 dense_30 (Dense)            (None, 930)               744930    
                                                                 
Total params: 1631330 (6.22 MB)
Trainable params: 1631330 (6.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
71/71 [==============================] - 1s 8ms/step - loss: 13.1502 - accuracy: 0.0280
Epoch 2/100
71/71 [==============================] - 1s 8ms/step - loss: 5.8237 - accuracy: 0.0449
Epoch 3/100
71/71 [==============================] - 1s 8ms/step - loss: 5.3528 - accuracy: 0.0520
Epoch 4/100
71/71 [==============================] - 1s 8ms/step - loss: 4.9482 - accuracy: 0.0747
Epoch 5/100
71/71 [==============================] - 1s 8ms/step - loss: 4.6453 - accuracy: 0.0840
Epoch 6/100
71/71 [==============================] - 1s 8ms/step - loss: 4.3898 - accuracy: 0.0911
Epoch 7/100
71/71 [==============================] - 1s 9ms/step - loss: 4.2094 - accuracy: 0.1138
Epoch 8/100
71/71 [==============================] - 1s 8ms/step - loss: 4.0354 - accuracy: 0.1276
Epoch 9/100
71/71 [==============================] - 1s 8ms/step - loss: 3.8683 - accuracy: 0.1373
Epoch 10/100
71/71 [==============================] - 1s 8ms/step - loss: 3.8180 - accuracy: 0.1369
Epoch 11

## Save Model for JS

In [22]:

output_dir = "./tfjs_models/FFNN1" 
tfjs.converters.save_keras_model(model, output_dir)

## Save word index

In [11]:
wordIndex = {}
for word in tokenizer.word_index:
    wordIndex[word] = tokenizer.word_index[word]

with open('./tokenizer/wordIndex.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [12]:
with open('./tokenizer/max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))